# ハミルトニアンを代数的に記述する

このチュートリアルでは、代数的モデラー`jijmodeling`を使ってQamomileのハミルトニアンを作成する方法を説明します。この機能を使うことで、より直感的にハミルトニアンを作成することが可能になります。

In [1]:
import jijmodeling as jm
import qamomile.core.modeler as qm_m

## 代数的ハミルトニアンの記述

このチュートリアルでは、周期的境界条件を持つ外場なしのハイゼンベルクモデルを、qamomileのモデラーを使って記述していきます。

パウリ演算子の式を定義するためには`qm_m.PauliExpr`を使用する必要があります。ハイゼンベルクモデルでは、$X$、$Y$、$Z$ のパウリ演算子が登場するため、まずはそれらを定義しましょう。

In [2]:
N = jm.Placeholder("N")
Z = qm_m.PauliExpr.z(shape=(N,))
X = qm_m.PauliExpr.x(shape=(N,))
Y = qm_m.PauliExpr.y(shape=(N,))
Y

BinaryVar(name='_PauliY', shape=[Placeholder(name='N', ndim=0)], latex='\hat{Y}', description='PauliY')

次に、以下のハミルトニアンを定義します：

$$
\hat{H} = - \sum_{i = 0}^{N-1} J_x X_i X_{i+1} + J_y Y_i Y_{i+1} + J_z Z_i Z_{i+1}
$$

ここでは周期的境界条件（$X_N = X_0$）を仮定しています。

このハミルトニアンは`jijmodeling`を使って定義することができます。

In [3]:
i = jm.Element("i", belong_to = (0,N))
J_x = jm.Placeholder("J_x")
J_y = jm.Placeholder("J_y")
J_z = jm.Placeholder("J_z")
expr = -jm.sum(i, J_x * X[i] * X[(i+1) % N] + J_y * Y[i] * Y[(i+1) % N] + J_z * Z[i] * Z[(i+1) % N])
h_expr = qm_m.HamiltonianExpr(expr)
h_expr

## 代数的なハミルトニアン式からQamomileのハミルトニアンを構築する

`HamiltonianExpr`に値を代入することで、Qamomileの`Hamiltonian`を構築できます。

まずは、代入するデータを作成しましょう。このデータは`dict`として提供することができます。

結果の出力が正しいかどうか確認しやすくするために、少し変わったデータを作成してみましょう。

In [4]:
instance_data = {"N":10, "J_x":1.0, "J_y":-1.0, "J_z":2.0}

`HamiltonianExpr`にインスタンスデータを挿入するには、`HamiltonianBuilder`を使用できます。

In [5]:
builder = qm_m.hamiltonian_expr.HamiltonianBuilder(h_expr,instance_data)
h = builder.build()
h

Hamiltonian((Z8, Z9): -2.0, (Z7, Z8): -2.0, (Z6, Z7): -2.0, (Z5, Z6): -2.0, (Z4, Z5): -2.0, (Z3, Z4): -2.0, (Z2, Z3): -2.0, (Z1, Z2): -2.0, (Z0, Z9): -2.0, (Z0, Z1): -2.0, (Y8, Y9): 1.0, (Y7, Y8): 1.0, (Y6, Y7): 1.0, (Y5, Y6): 1.0, (Y4, Y5): 1.0, (Y3, Y4): 1.0, (Y2, Y3): 1.0, (Y1, Y2): 1.0, (Y0, Y9): 1.0, (Y0, Y1): 1.0, (X8, X9): -1.0, (X7, X8): -1.0, (X6, X7): -1.0, (X5, X6): -1.0, (X4, X5): -1.0, (X3, X4): -1.0, (X2, X3): -1.0, (X1, X2): -1.0, (X0, X9): -1.0, (X0, X1): -1.0)

`Hamiltonian`をLaTeX形式で確認したい場合は、次のようにできます。


In [6]:
import IPython.display as ipd

ipd.display(ipd.Latex("$" + h.to_latex() + "$"))

<IPython.core.display.Latex object>